In [1]:
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted rest.ipynb.
Converted services.ipynb.


In [2]:
# default_exp rest

In [3]:
#export
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property

from flask import Flask
from flask_restplus import Resource, Api, fields, reqparse
from pathlib import Path
from services import Scrum
from contextlib import contextmanager

In [4]:
#export
app = Flask(__name__)
api = Api(app)
ns = api.namespace('quick-scrum', description='Quick Scrum API')
scrum = Scrum('../data')

In [5]:
@api.errorhandler(ValueError)
def handle_value_error(error):
    '''Return a custom message and 400 status code'''
    return {'message': error.args[0]}, 400


In [6]:
#export

project_model = api.model('Project', {
    'name': fields.String(required=True),
    'description': fields.String
})

@api.route('/api/v1/projects')
class ProjectList(Resource):
    @ns.doc('get list of projects')
    def get(self):
        return scrum.get_projects()

    @ns.doc('create a new project')
    @ns.expect(project_model)
    def post(self):
        p = api.payload
        n = p['name']
        scrum.new_project(p)
        return f'Project {n} created', 201

In [7]:
#export

@api.route('/api/v1/projects/<string:project_name>', endpoint='project')
@api.doc(params={
    'project_name': 'Id of a project',
    'log': 'The log where the stories are placed, sandbox, backlog, sprint-1, sprint-2...'
})
class Project(Resource):
    @ns.doc('get list of projects')
    def get(self, project_name):
        return {
            'name': project_name,
            'sprints': scrum.get_sprints(project_name)
        }

In [8]:
#export

story_model = api.model('Project', {
    'name': fields.String(required=True),
    'description': fields.String,
    'points': fields.Integer,
    'state': fields.String,
    'tasks': fields.List(fields.Nested({
        'description': fields.String(required=True),
        'start': fields.Date,
        'end': fields.Date,
        'owner': fields.String
    }))
})

def is_valid_store(store):
    return store in ['sandbox', 'backlog'] or log.startswith('sprint-')
    
def invalid_store(store):
    return f'Invalid store {store}. Valid stores are backlog, sandbox and sprint-[n]', 400
    
@api.route('/api/v1/projects/<string:project_name>/<string:store>', endpoint='stories')
@api.doc(params={
    'project_name': 'Id of a project',
    'store': 'The store where the stories are placed, sandbox, backlog, sprint-1, sprint-2...'
})
class StoryList(Resource):
    @ns.doc('get stories')
    def get(self, project_name: str, store: str):
        if not is_valid_store(store): return invalid_store(store)
        return scrum.get_stories(project_name, store)

    @ns.doc('create a new story')
    def post(self, project_name: str, store: str):
        parser = reqparse.RequestParser()
        parser.add_argument('move_from', type=str, help='Original story location')
        args = parser.parse_args()
        move_from = args['move_from']
        
        if move_from != None:
            source,name = move_from.split('/')
            if scrum.move_story(project_name, source, store, name):
                return f'{target}/{name}', 201
            else:
                return f'Internal error', 500
        else:
            scrum.set_story(project_name, store, api.payload)
            return '', 201


@api.route('/api/v1/projects/<string:project_name>/<string:store>/<string:story_name>', endpoint='story')
@api.doc(params={
    'project_name': 'Id of a project',
    'log': 'The log where the stories are placed, sandbox, backlog, sprint-1, sprint-2...',
    'story_name': 'Story id'
})
class Story(Resource):
    @ns.doc('get a story')
    def get(self, project_name: str, store: str, story_name: str):
        return scrum.get_story(project_name, store, story_name)

        

In [ ]:
#hide
app.run(port=4444,debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4444/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Sep/2020 17:55:57] "GET /api/v1/projects/test2/sandbox/test2 HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2020 17:56:24] "POST /api/v1/projects/test2/sandbox HTTP/1.1" 201 -
127.0.0.1 - - [28/Sep/2020 17:56:42] "GET /api/v1/projects/test2/sandbox/test3 HTTP/1.1" 200 -
